In [ ]:
!git clone https://github.com/companys1234/Jumping_LLM_Flash.git

Cloning into 'Jumping_LLM_Flash'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 147 (delta 42), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 58.17 KiB | 416.00 KiB/s, done.
Resolving deltas: 100% (42/42), done.


In [ ]:
from Jumping_LLM_Flash.scr.preprocessing import SimpleTokenizer
from Jumping_LLM_Flash.scr.architecture import *
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import math

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-01 15:56:38--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  6.27MB/s    in 0.2s    

2026-01-01 15:56:39 (6.27 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
tokenizer = SimpleTokenizer(text)

In [ ]:
def get_batch(data, block_size, batch_size, device):
    """
    data: torch.Tensor (1D)
    block_size: int — длина контекста
    """
    ix = torch.randint(0, len(data) - block_size - 1, (batch_size,), device=device)

    x = torch.stack([data[i:i + block_size] for i in ix])  # (B, block_size)
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])  # (B, block_size)

    return x, y

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
vocab_size = 69
class llama2(nn.Module):
  def __init__(self):
    super().__init__()
    self.rmsnorm = RMSNorm(128)
    self.rope = RoPE
    self.embedding = nn.Embedding(vocab_size, 128)
    #self.swiglu = SwigLU
    self.GMQA = GMQA_with_KV(128,4,1)
    self.rmsnorm2 = RMSNorm(128*2)
    self.ff = Feed_Forward_Network(128 * 2, 200, SwigLU())
    self.rms3 = RMSNorm(128 * 3)  # после второй конкатенации
    self.out_proj = nn.Linear(128 * 3, vocab_size)
  def forward(self,x):
        x= self.embedding(x)
        gmqa_out, attn_weights, present_kv = self.GMQA(x,True)

        concat1 = torch.cat([x, gmqa_out], dim=-1)
        concat1_norm = self.rmsnorm2(concat1)
        ff_out = self.ff(concat1_norm)

        concat2 = torch.cat([gmqa_out, ff_out], dim=-1)
        concat2_norm = self.rms3(concat2)

        logits = self.out_proj(concat2_norm)
        return logits, present_kv

  @torch.no_grad()
  def generate(self, idx, max_new_tokens):
        #past_kv = None
        for _ in range(max_new_tokens):
            logits = self.forward(idx)[0]
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_token], dim=1)
        return idx

In [ ]:
def train(model, data, optimizer, criterion, device, epochs=42, batch_size=8, block_size=8):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for _ in range(len(data) // batch_size):
            ix = torch.randint(0, len(data) - block_size - 1, (batch_size,), device=device)
            x = torch.stack([data[i:i + block_size] for i in ix])
            y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])

            optimizer.zero_grad()
            logits, _ = model(x)

            loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / (len(data) // batch_size)
        print(f"Epoch {epoch+1}: loss={avg_loss:.4f}")

In [ ]:
text.lower()
tokens = torch.tensor(tokenizer.encode(text[:10000]), dtype=torch.long)

/tmp/ipython-input-3419825771.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.tensor(tokenizer.encode(text[:6000]), dtype=torch.long)


In [ ]:
# создаём модель
model = llama2()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
train(
    model=model,
    data=tokens,
    optimizer=optimizer,
    criterion=criterion,
    device = 'cpu'
)

Epoch 1: loss=1.7163
Epoch 2: loss=1.3028
Epoch 3: loss=1.1708
Epoch 4: loss=1.0772
Epoch 5: loss=1.0060
Epoch 6: loss=0.9518
Epoch 7: loss=0.8939
Epoch 8: loss=0.8525
Epoch 9: loss=0.7981
Epoch 10: loss=0.7669
Epoch 11: loss=0.7367
Epoch 12: loss=0.7133
Epoch 13: loss=0.6882
Epoch 14: loss=0.6613
Epoch 15: loss=0.6305
Epoch 16: loss=0.6302
Epoch 17: loss=0.6142
Epoch 18: loss=0.5985
Epoch 19: loss=0.5756
Epoch 20: loss=0.5706
Epoch 21: loss=0.5637
Epoch 22: loss=0.5432
Epoch 23: loss=0.5414
Epoch 24: loss=0.5286
Epoch 25: loss=0.5187
Epoch 26: loss=0.5219
Epoch 27: loss=0.5072
Epoch 28: loss=0.4984
Epoch 29: loss=0.4870
Epoch 30: loss=0.4794
Epoch 31: loss=0.4830
Epoch 32: loss=0.4786
Epoch 33: loss=0.4717
Epoch 34: loss=0.4592
Epoch 35: loss=0.4571
Epoch 36: loss=0.4488
Epoch 37: loss=0.4544
Epoch 38: loss=0.4405
Epoch 39: loss=0.4375
Epoch 40: loss=0.4359
Epoch 41: loss=0.4357
Epoch 42: loss=0.4297


In [ ]:
def chat(model, tokenizer, prompt, device, max_new_tokens):
    model.eval()
    input_ids = torch.tensor(tokenizer.encode(prompt), dtype=torch.long, device=device)
    input_ids = input_ids.unsqueeze(0)

    with torch.no_grad():
        output_ids = model.generate(input_ids, max_new_tokens=max_new_tokens)

    output_text = tokenizer.decode(output_ids[0].tolist())
    return output_text

In [ ]:
vocab = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,!?:;'\n-&3$#@э!?*")

vocab_size = 69

prompt = "First Citizen: We are accounted poor citizens, the patricians good. What authority surfeits on would relieve us: if they"

response = chat(model, tokenizer, prompt, device='cpu', max_new_tokens=50)
print('response:',response)

response: First Citizen: We are accounted poor citizens, the patricians good. What authority surfeits on would relieve us: if theyouce coursthe corithe corthe couche coucouche coun


/tmp/ipython-input-2681672045.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(tokenizer.encode(prompt), dtype=torch.long, device=device)
